# Modell-Typen im Überblick

In diesem Notebook lernen wir verschiedene Typen von Large Language Models (LLMs) kennen und erkunden ihre spezifischen Eigenschaften und Anwendungsfälle.


## 1. LLM-Modelle im Vergleich

Wir beginnen mit einem Überblick der gängigsten LLM-Modelle und ihrer Eigenschaften.

In [ ]:
from helpers import llm
import time

# Standard-Modell ist gpt-4o
standard_model = llm()

# Verschiedene Modelle im Vergleich
models = {
    "gpt-3.5-turbo": llm(model="gpt-3.5-turbo"),
    "gpt-4o": llm(model="gpt-4o"),
    "gpt-4o-mini": llm(model="gpt-4o-mini"),
}

test_prompt = "Erkläre mir das Konzept einer rekursiven Funktion in 2-3 Sätzen."

# Vergleich von Antwortzeiten und Qualität
for model_name, model_instance in models.items():
    print(f"\n--- Modell: {model_name} ---")
    start_time = time.time()
    response = model_instance.invoke(test_prompt)
    end_time = time.time()
    
    print(f"Antwort: {response.content}")
    print(f"Dauer: {end_time - start_time:.2f} Sekunden")

## 2. Embeddings: Grundlagen und Anwendungen

Embeddings sind numerische Vektorrepräsentationen von Text, Bildern oder anderen Daten. Sie bilden semantische Eigenschaften in einem mehrdimensionalen Raum ab und sind fundamental für viele KI-Anwendungen.

In [ ]:
import tiktoken

# Tokenisierung verstehen
encoding = tiktoken.encoding_for_model("gpt-4o")

example_text = "Künstliche Intelligenz verändert die Welt."
tokens = encoding.encode(example_text)
print(f"Text: '{example_text}'")
print(f"Token-IDs: {tokens}")

# Jeder Token einzeln dekodiert
decoded_tokens = [encoding.decode_single_token_bytes(token).decode("utf-8") for token in tokens]
print("\nToken für Token:")
for i, token in enumerate(decoded_tokens):
    print(f"Token {i+1}: '{token}'")

## Embeddings mit OpenAI erzeugen und vergleichen

Embeddings ermöglichen es uns, semantische Ähnlichkeiten zwischen Texten zu berechnen. Wir werden einige Beispielsätze vergleichen.

In [ ]:
from openai import OpenAI
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Embeddings für verschiedene Sätze berechnen
texts = [
    "Künstliche Intelligenz verändert die Welt",
    "KI revolutioniert alle Wirtschaftsbereiche",
    "Machine Learning ist ein Teilbereich der Künstlichen Intelligenz",
    "Katzen sind niedliche Haustiere",
    "Hunde werden oft als treue Begleiter bezeichnet",
    "Tiere spielen eine wichtige Rolle im Ökosystem",
]

response = client.embeddings.create(
    input=texts,
    model="text-embedding-ada-002"
)

# Embeddings extrahieren
embeddings = [np.array(item.embedding) for item in response.data]

# Cosinus-Ähnlichkeit berechnen
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Ähnlichkeitsmatrix erstellen
similarity_matrix = np.zeros((len(texts), len(texts)))
for i in range(len(texts)):
    for j in range(len(texts)):
        similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])

# Visualisierung der Ähnlichkeitsmatrix
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='viridis')
plt.colorbar(label='Cosinus-Ähnlichkeit')
plt.xticks(range(len(texts)), [f"Text {i+1}" for i in range(len(texts))], rotation=45)
plt.yticks(range(len(texts)), [f"Text {i+1}" for i in range(len(texts))])
plt.title('Semantische Ähnlichkeit zwischen Texten')

# Beschriftung für die Texte
for i in range(len(texts)):
    print(f"Text {i+1}: {texts[i]}")

plt.tight_layout()
plt.show()

# Visualisierung mit PCA für 2D-Darstellung
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 8))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], s=100)

for i, (x, y) in enumerate(reduced_embeddings):
    plt.annotate(f"Text {i+1}", (x, y), xytext=(5, 5), textcoords='offset points')

plt.title('2D-Visualisierung der Embeddings')
plt.xlabel('Erste Hauptkomponente')
plt.ylabel('Zweite Hauptkomponente')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 3. Multimodale Modelle: Text und Bild

Multimodale Modelle können verschiedene Arten von Eingaben (Text, Bild, Audio) verarbeiten und miteinander verknüpfen. Hier ein Beispiel für die Arbeit mit Bildern in GPT-4o.

In [ ]:
from langchain.schema import HumanMessage

# GPT-4o als multimodales Modell verwenden
vision_llm = llm(model="gpt-4o", max_tokens=1024)

# Beispiel: Bildanalyse mit einem Comic
input = [
    HumanMessage(
        content=[
            "Was ist auf diesem Bild zu sehen? Erkläre auch den Witz dahinter.",
            {"image_url": "https://joscha.com/data/media/cartoons/130608.png"},
        ]
    )
]

response = vision_llm.invoke(input)
print(response.content)

## Praktische Anwendung: Bild generieren und analysieren

Wir kombinieren nun Text-zu-Bild-Generierung mit Bildanalyse, um den vollständigen multimodalen Kreislauf zu demonstrieren.

In [ ]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from IPython.display import Image, display
import requests
from io import BytesIO

# 1. Prompt für die Bildgenerierung erstellen
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            "Generiere einen präzisen Prompt für DALL-E, um ein Bild zu erzeugen, das folgendes Konzept visualisiert: {image_desc}"
        )
    ]
)

# 2. Chain für die Prompt-Optimierung
chain = prompt | llm(temperature=0.7) | StrOutputParser()

# 3. Nutzereingabe für das Bild
user_image_description = "Ein Roboter, der in einer Bibliothek sitzt und Bücher liest, während er gleichzeitig neue Bücher schreibt"

# 4. Optimierten Prompt generieren
dalle_prompt = chain.invoke({"image_desc": user_image_description})
print(f"Optimierter DALL-E Prompt:\n{dalle_prompt}\n")

# 5. Bild mit DALL-E generieren
try:
    image_url = DallEAPIWrapper(model="dall-e-2", size="1024x1024").run(dalle_prompt)
    print(f"Generiertes Bild: {image_url}")
    
    # Bild herunterladen und anzeigen
    response = requests.get(image_url)
    img = Image(data=response.content)
    display(img)
    
    # 6. Bild mit GPT-4o analysieren
    vision_input = [
        HumanMessage(
            content=[
                "Beschreibe detailliert, was auf diesem Bild zu sehen ist. Wie gut repräsentiert es das Konzept: '" + user_image_description + "'?",
                {"image_url": image_url},
            ]
        )
    ]
    
    vision_response = vision_llm.invoke(vision_input)
    print("\nAnalyse des generierten Bildes:")
    print(vision_response.content)
    
except Exception as e:
    print(f"Fehler bei der Bildgenerierung oder -analyse: {e}")
    print("Um diesen Teil auszuführen, stellen Sie sicher, dass Sie einen gültigen DALL-E API-Schlüssel haben.")

## 4. Reranker für verbesserte Textsuche (optionales Thema)

Reranker verbessern die Qualität von Suchergebnissen durch eine präzisere Bewertung der Relevanz zwischen einer Anfrage und potenziellen Dokumenten.

In [ ]:
# Beispiel-Code für Reranking mit Sentence Transformers
# Wenn Sie diesen Code ausführen möchten, installieren Sie: pip install sentence-transformers

try:
    from sentence_transformers import CrossEncoder
    
    # Beispiel-Dokumente
    documents = [
        "Berlin ist die Hauptstadt von Deutschland und hat etwa 3,7 Millionen Einwohner.",
        "Paris ist die Hauptstadt von Frankreich und ein beliebtes Reiseziel.",
        "Der Fluss Spree fließt durch Berlin und ist wichtig für die Stadt.",
        "Berlin hat viele Sehenswürdigkeiten wie das Brandenburger Tor.",
        "Deutschland grenzt an neun Nachbarländer in Europa."
    ]
    
    # Abfrage
    query = "Was ist die Hauptstadt von Deutschland?"
    
    print("Abfrage:", query)
    print("\nDokumente ohne Ranking:")
    for i, doc in enumerate(documents):
        print(f"{i+1}. {doc}")
    
    # Reranking mit einem Cross-Encoder
    reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[query, doc] for doc in documents]
    scores = reranker.predict(pairs)
    
    # Ergebnisse sortieren
    ranked_results = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    
    print("\nDokumente nach Reranking:")
    for i, (doc, score) in enumerate(ranked_results):
        print(f"{i+1}. Score: {score:.4f} - {doc}")
        
except ImportError:
    print("Sentence Transformers nicht installiert. Bitte führen Sie 'pip install sentence-transformers' aus, um dieses Beispiel zu nutzen.")

## 5. Modellauswahl in der Praxis

Lassen Sie uns das LLM selbst nach Entscheidungskriterien für die Modellauswahl fragen.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# Prompt für Modellauswahl-Empfehlungen
model_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein erfahrener KI-Architekt, der Teams bei der Auswahl des richtigen LLM-Modells für ihre Anwendungen berät."),
    ("human", """
    Erstelle eine Entscheidungsmatrix für folgende Anwendungsfälle und empfehle das optimale Modell:
    
    1. Ein Chatbot für einfache Kundenanfragen auf einer E-Commerce-Website
    2. Ein System zur Analyse und Zusammenfassung wissenschaftlicher Fachartikel
    3. Eine Anwendung zur Analyse von Röntgenbildern mit begleitendem Befundtext
    4. Ein Tool für die automatische Codegenerierung für Entwickler
    5. Ein Übersetzungssystem für juristische Texte, das in einer Bank ohne Internetverbindung laufen soll
    
    Berücksichtige folgende Faktoren: Kosten, Leistung, Datenschutz, Multimodale Fähigkeiten und Integrationsaufwand.
    Strukturiere deine Antwort übersichtlich in Tabellenform und füge eine kurze Begründung für jede Empfehlung hinzu.
    """)
])

model_selection_chain = model_selection_prompt | llm() | StrOutputParser()
response = model_selection_chain.invoke({})
print(response)

## 6. Praktische Übungen

### Übung 1: Token-Analyse
Analysieren Sie, wie verschiedene Texte in Tokens zerlegt werden.

In [ ]:
def analyze_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(text)
    decoded_tokens = [encoding.decode_single_token_bytes(token).decode("utf-8") for token in tokens]
    
    print(f"Text: '{text}'")
    print(f"Anzahl Tokens: {len(tokens)}")
    print("Tokens:")
    for i, token in enumerate(decoded_tokens):
        print(f"  {i+1}: '{token}'")
    print("\n")

# Beispiel
analyze_tokens("Künstliche Intelligenz ist faszinierend!")

# Übung: Analysieren Sie verschiedene Texte
# Beispiel 1: Fachbegriffe
analyze_tokens("Machine Learning und Deep Learning sind Teilbereiche der KI.")

# Beispiel 2: Sonderzeichen und Zahlen
analyze_tokens("Das kostet 123,45€ und ist verfügbar ab dem 01.01.2023!")

# Beispiel 3: Eigener Text (hier einfügen)
# analyze_tokens("Ihr eigener Text hier...")

### Übung 2: Embedding-Vergleich verschiedener Konzepte

Erstellen Sie Embeddings für verschiedene Konzepte und vergleichen Sie die Ähnlichkeiten.

In [ ]:
# Eigene Texte für Embedding-Vergleich definieren
your_texts = [
    # Fügen Sie hier 4-6 verschiedene Texte oder Konzepte ein
    # Beispiel:
    "Maschinelles Lernen ist ein Teilgebiet der künstlichen Intelligenz",
    "Data Science umfasst Statistik, Programmierung und Domänenwissen",
    "Python ist eine beliebte Programmiersprache für KI-Anwendungen",
    "Neuronale Netze sind vom menschlichen Gehirn inspiriert",
    # Fügen Sie weitere Texte hinzu...
]

# Embeddings berechnen (wenn OpenAI API verfügbar ist)
try:
    response = client.embeddings.create(
        input=your_texts,
        model="text-embedding-ada-002"
    )
    
    embeddings = [np.array(item.embedding) for item in response.data]
    
    # Ähnlichkeitsmatrix
    similarity_matrix = np.zeros((len(your_texts), len(your_texts)))
    for i in range(len(your_texts)):
        for j in range(len(your_texts)):
            similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])
    
    # Visualisierung
    plt.figure(figsize=(10, 8))
    plt.imshow(similarity_matrix, cmap='viridis')
    plt.colorbar(label='Cosinus-Ähnlichkeit')
    plt.xticks(range(len(your_texts)), [f"Text {i+1}" for i in range(len(your_texts))], rotation=45)
    plt.yticks(range(len(your_texts)), [f"Text {i+1}" for i in range(len(your_texts))])
    plt.title('Ihre semantische Ähnlichkeitsmatrix')
    
    # Textbeschreibungen
    for i, text in enumerate(your_texts):
        print(f"Text {i+1}: {text}")
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Fehler bei der Embedding-Berechnung: {e}")
    print("Stellen Sie sicher, dass die OpenAI API korrekt konfiguriert ist.")

## 7. Zusammenfassung

In diesem Notebook haben wir folgende Konzepte kennengelernt:

1. **Verschiedene LLM-Modelle** und ihre spezifischen Stärken und Schwächen
2. **Embeddings** als Vektorrepräsentationen von Text und ihre Anwendungen in semantischer Suche
3. **Multimodale Modelle** wie GPT-4o, die Text und Bilder verarbeiten können
4. **Reranker** zur Verbesserung von Suchergebnissen (optionales Thema)
5. **Praktische Entscheidungskriterien** für die Modellauswahl in realen Projekten

Diese Grundlagen bilden die Basis für fortgeschrittene KI-Anwendungen und geben einen Überblick über die vielfältigen Möglichkeiten moderner LLMs.